In [1]:
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np
import os.path
from tqdm import tqdm
import csv
import random
import glob
import os.path
import sys
import operator
import threading
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

Using TensorFlow backend.


In [2]:
seq_length= 40
max_frames = 300
image_shape=(224, 224, 3)
base_path = "RNN-Video"


In [3]:
with open(os.path.join("RNN-Video/data_file_5class.csv"), "r") as fin:
    reader = csv.reader(fin)
    data = list(reader)

In [4]:
train_path = os.path.join(base_path, "train")
classes =os.listdir(train_path)
classes = sorted(classes)
classes

['CricketBowling',
 'CricketShot',
 'FieldHockeyPenalty',
 'HandstandPushups',
 'HandstandWalking',
 'SoccerPenalty']

In [6]:
data_clean = []
for item in data:
    if int(item[3]) >= seq_length and int(item[3]) <= max_frames:
        data_clean.append(item)

In [7]:
len(data_clean)

439

In [8]:
def get_n_sample_from_video(sample, seq_length):
    path = os.path.join(base_path, sample[0], sample[1])
    filename = sample[2]
    images = sorted(glob.glob(os.path.join(path, filename + "*jpg")))
    skip = len(images) // seq_length
    output = [images[i] for i in range(0, len(images), skip)]
    return output[:seq_length]

In [9]:
data_clean[3]

['train', 'HandstandWalking', 'v_HandstandWalking_g24_c06', '151']

In [10]:
len(get_n_sample_from_video(data_clean[3], 40))

40

In [11]:
base_model = InceptionV3(weights= "imagenet", include_top=True)
model = Model(inputs=base_model.input,
        outputs=base_model.get_layer("avg_pool").output)

def model_predict(image_path):
    img = image.load_img(image_path, target_size= (299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis= 0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features[0]

In [12]:
os.makedirs("sequences", exist_ok=True)
for video in tqdm(data_clean):
    path = os.path.join("sequences", video[2] + "-" + str(seq_length) + "-features")  
    if os.path.isfile(path + ".npy"):
        continue

    frames = get_n_sample_from_video(video, seq_length)
    sequence = []
    for frame in frames:
        features = model_predict(frame)
        sequence.append(features)
    np.save(path, sequence)

100%|████████████████████████████████████████████████████████████████████████████████| 439/439 [18:31<00:00,  2.85s/it]
